In [2]:
import pandas as pd
import numpy as np
import requests
from time import sleep
from pandas.io.json import json_normalize
import timeit
from __future__ import division
import time

**Inpurts are congress number, and bill to start from. Bill 1 does not always exist**

In [12]:
%%time

bill_url = "https://www.govtrack.us/data/congress/%d/bills/hr/hr%d/data.json" %(114, 1)

# get raw data on a bill
r = requests.get(bill_url)

# check the status code
# if 404 then there is no data for that bill
# if 200 then there is data for the bill. Collect it and save it as an object
if r.status_code == 404:
    print "no such data for hr%d" %1
    bad_request_count = 1
if r.status_code == 200:
    print "Alex, you da man! Searching for hr%d" %1
    bad_request_count = 0
    json_pol = r.json()
    x = json_normalize(json_pol)

# look within the nested data to see if a 'roll' key exists
# if a roll key exists then the bill was voted on
# if the roll key does not exist then there was no vote, 
# therefore we do not care about the data
    try:
        x_1 = json_normalize(x['actions'][0])
#deleted
        if 'roll' in x_1:
            x_to_append = pd.DataFrame(x_1[x_1['roll'] > 0]).reset_index(drop=True)
            x_to_append = pd.DataFrame(x_to_append[x_to_append['where'] == "h"][['roll', 'where', 'acted_at']]).reset_index(drop=True)
            x_to_append['date'] = x_to_append['acted_at'].str[:4]

            # append the roll call number to the df
            # and subset the original raw data for the desired features
            x['roll'] = x_to_append['roll']
            x['where'] = 'h'
            df = x[['bill_id', 'bill_type', 'congress', 'official_title', 'popular_title', 
                    'short_title', 'sponsor.state','sponsor.title', 'sponsor.type', 'subjects',
                    'subjects_top_term', 'summary.date', 'roll', 'where']]


            # use the roll call number to collect raw data for voting on a bill
            # there may be more than one roll call number
            # if there is only one roll call number it's easy!
            # just collect the number and do all your data scrubbing for that bill
            if x_to_append.shape[0] == 1:
                try:
                    url = "https://www.govtrack.us/data/congress/114/votes/%d/h%d/data.json" %(x['summary.date'].str[:4], int(x_to_append['roll'][0]))
                except KeyError:
                    print "['summary.date'] not in index"
                    try:
                        url = "https://www.govtrack.us/data/congress/114/votes/%d/h%d/data.json" %(int(x_to_append['date'][0]), int(x_to_append['roll'][0]))
                    except KeyError:
                        print "['summary.date'] not in index"

                r = requests.get(url)
                json_pol = r.json()
                roll_call_data = json_normalize(json_pol)


                # extract 'Nay', 'Yea', and 'Not Voting' within raw roll call data
                if 'votes.No' in roll_call_data:
                    nay = pd.DataFrame(roll_call_data['votes.No'][0])
                if 'votes.Aye' in roll_call_data:
                    yea = pd.DataFrame(roll_call_data['votes.Aye'][0])
                if 'votes.Not Voting' in roll_call_data:
                    not_voting = pd.DataFrame(roll_call_data['votes.Not Voting'][0])
                nay['vote'] = 'no'
                yea['vote'] = 'yea'
                not_voting['vote'] = 'not voting'


                # append 'Nay', 'Yea', and 'Not Voting' to a single dataframe

                bill_votes = nay.append([yea, not_voting])

                # Make a single dataframe will relevant information

                finalized_dataframe = bill_votes
                finalized_dataframe['bill_id'] = df['bill_id'][0]
                finalized_dataframe['subjects_top_terms'] = df['subjects_top_term'][0]
                finalized_dataframe['subjects'] = str(df['subjects'][0])
                finalized_dataframe['date'] = df['summary.date'][0]

            # if there is more than one roll call number then you're fucked
            # lol, actually not fucked, its just more difficult
            # now you have to create another loop and do the data scrubbing for one bill
            # multiple times
            if x_to_append.shape[0] > 1:
                for j in range(0, x_to_append.shape[0]):
                    try:
                        url = "https://www.govtrack.us/data/congress/114/votes/%d/h%d/data.json" %(x['summary.date'].str[:4], int(x_to_append['roll'][j]))
                    except KeyError:
                        print "['summary.date'] not in index"
                        try:
                            url = "https://www.govtrack.us/data/congress/114/votes/%d/h%d/data.json" %(int(x_to_append['date'][j]), int(x_to_append['roll'][j]))
                        except KeyError:
                            print "['summary.date'] not in index"

                    r = requests.get(url)
                    json_pol = r.json()
                    roll_call_data = json_normalize(json_pol)


                    # extract 'Nay', 'Yea', and 'Not Voting' within raw roll call data

                    if 'votes.No' in roll_call_data:
                        nay = pd.DataFrame(roll_call_data['votes.No'][0])
                        nay['vote'] = 'no'
                    if 'votes.Aye' in roll_call_data:
                        yea = pd.DataFrame(roll_call_data['votes.Aye'][0])
                        yea['vote'] = 'yea'
                    if 'votes.Not Voting' in roll_call_data:
                        not_voting = pd.DataFrame(roll_call_data['votes.Not Voting'][0])
                        not_voting['vote'] = 'not voting'

                    # append 'Nay', 'Yea', and 'Not Voting' to a single dataframe

                    bill_votes = nay.append([yea, not_voting])
                    bill_votes = bill_votes[['display_name', 'id', 'party', 'state', 'vote']]

                    # Make a single dataframe will relevant information

                    finalized_dataframe = bill_votes
                    finalized_dataframe['bill_id'] = df['bill_id'][0]
                    finalized_dataframe['subjects_top_terms'] = df['subjects_top_term'][0]
                    finalized_dataframe['subjects'] = str(df['subjects'][0])
                    finalized_dataframe['date'] = df['summary.date'][0]

        else:
            print "no roll data for hr%d" %1

    except IndexError:
        print "list index out of range"
    # Start loop to collect and append data
    # Start loop to collect and append data
    # Start loop to collect and append data
    # Start loop to collect and append data

    for i in range(1339,7000):
        #create dynamic variable to call gov track api and get bill data
        # the hr%d will increase because it is the bill number
        # eventually it would be nice to get the congress number to be dynamic 
            ## although idk if I need that
        bill_url = "https://www.govtrack.us/data/congress/%d/bills/hr/hr%d/data.json" %(114, i)

        # get raw data on a bill
        try:
            r = requests.get(bill_url)
        except requests.ConnectionError:
            print "must sleep for 10 seconds"
            time.sleep(10)
            try:
                r = requests.get(bill_url)
            except requests.ConnectionError:
                print "must sleep for 1 minute"
                time.sleep(60)
                r = requests.get(bill_url)
            

        # check the status code
        # if 404 then there is no data for that bill
        # if 200 then there is data for the bill. Collect it and save it as an object
        if r.status_code == 404:
            print "no such data for hr%d" %i
            if bad_request_count > 0:
                bad_request_count = bad_request_count + 1
                if bad_request_count == 25:
                    print "no more bills made"
                    break
            if bad_request_count == 0:
                bad_request_count = 1
        if r.status_code == 200:
            print "Alex, you da man! Searching for hr%d" %i
            bad_request_count = 0
            json_pol = r.json()
            x = json_normalize(json_pol)

        # look within the nested data to see if a 'roll' key exists
        # if a roll key exists then the bill was voted on
        # if the roll key does not exist then there was no vote, 
        # therefore we do not care about the data
            try:
                x_1 = json_normalize(x['actions'][0])
#            except IndexError:
#                print "list index out of range"
                if 'roll' in x_1:
                    x_to_append = pd.DataFrame(x_1[x_1['roll'] > 0]).reset_index(drop=True)
                    x_to_append = pd.DataFrame(x_to_append[x_to_append['where'] == "h"][['roll', 'where', 'acted_at']]).reset_index(drop=True)
                    x_to_append['date'] = x_to_append['acted_at'].str[:4]
                    # append the roll call number to the df
                    # and subset the original raw data for the desired features

                    x['roll'] = x_to_append['roll']
                    x['where'] = 'h'
                    try: 
                        df_loop = x[['bill_id', 'bill_type', 'congress', 'official_title', 'popular_title', 
                                'short_title', 'sponsor.state','sponsor.title', 'sponsor.type', 'subjects',
                                'subjects_top_term', 'summary.date', 'roll', 'where']]


                        # use the roll call number to collect raw data for voting on a bill
                        # there may be more than one roll call number
                        # if there is only one roll call number it's easy!
                        # just collect the number and do all your data scrubbing for that bill
                        if x_to_append.shape[0] == 1:
                            try:
                                url = "https://www.govtrack.us/data/congress/114/votes/%d/h%d/data.json" %(x['summary.date'].str[:4], int(x_to_append['roll'][0]))
                            except Exception:
                                print "['summary.date'] not in index"
                                try:
                                    url = "https://www.govtrack.us/data/congress/114/votes/%d/h%d/data.json" %(int(x_to_append['date'][0]), int(x_to_append['roll'][0]))
                                except Exception:
                                    print "['date'] not in index"


                            r = requests.get(url)
                            try:
                                json_pol = r.json()
                            except ValueError:
                                print "No JSON object could be decoded"
                            except requests.ConnectionError:
                                print "Connection error"
                            roll_call_data = json_normalize(json_pol)


                            # extract 'Nay', 'Yea', and 'Not Voting' within raw roll call data

                            if 'votes.No' in roll_call_data:
                                nay = pd.DataFrame(roll_call_data['votes.No'][0])
                                nay['vote'] = 'no'
                            if 'votes.Aye' in roll_call_data:
                                yea = pd.DataFrame(roll_call_data['votes.Aye'][0])
                                yea['vote'] = 'yea'
                            if 'votes.Not Voting' in roll_call_data:
                                not_voting = pd.DataFrame(roll_call_data['votes.Not Voting'][0])
                                not_voting['vote'] = 'not voting'

                            # append 'Nay', 'Yea', and 'Not Voting' to a single dataframe

                            bill_votes_loop = nay.append([yea, not_voting])

                            # Make a single dataframe will relevant information
                            finalized_dataframe_loop = bill_votes_loop
                            finalized_dataframe_loop['bill_id'] = df_loop['bill_id'][0]
                            finalized_dataframe_loop['subjects_top_terms'] = df_loop['subjects_top_term'][0]
                            finalized_dataframe['subjects'] = str(df['subjects'][0])
                            finalized_dataframe_loop['date'] = df_loop['summary.date'][0]

                            df = df.append(df_loop)
                            bill_votes = bill_votes.append(bill_votes_loop)
                            finalized_dataframe = finalized_dataframe.append(finalized_dataframe_loop)

                            # if there is more than one roll call number then you're fucked
                            # lol, actually not fucked, its just more difficult
                            # now you have to create another loop and do the data scrubbing for one bill
                            # multiple times
                            if x_to_append.shape[0] > 1:
                                for j in range(0, x_to_append.shape[0]):
                                    print "This bill has multiple votes, We are on vote %d" %j
                                    try:
                                        url = "https://www.govtrack.us/data/congress/114/votes/%d/h%d/data.json" %(x['summary.date'].str[:4], int(x_to_append['roll'][j]))
                                    except KeyError:
                                        print "['summary.date'] not in index"
                                        try:
                                            url = "https://www.govtrack.us/data/congress/114/votes/%d/h%d/data.json" %(int(x_to_append['date'][j]), int(x_to_append['roll'][j]))
                                        except KeyError:
                                            print "['summary.date'] not in index"

                                    r = requests.get(url)
                                    json_pol = r.json()
                                    roll_call_data = json_normalize(json_pol)


                                    # extract 'Nay', 'Yea', and 'Not Voting' within raw roll call data

                                    if 'votes.No' in roll_call_data:
                                        nay = pd.DataFrame(roll_call_data['votes.No'][0])
                                        nay['vote'] = 'no'
                                    if 'votes.Aye' in roll_call_data:
                                        yea = pd.DataFrame(roll_call_data['votes.Aye'][0])
                                        yea['vote'] = 'yea'
                                    if 'votes.Not Voting' in roll_call_data:
                                        not_voting = pd.DataFrame(roll_call_data['votes.Not Voting'][0])
                                        not_voting['vote'] = 'not voting'

                                    # append 'Nay', 'Yea', and 'Not Voting' to a single dataframe

                                    bill_votes_loop = nay.append([yea, not_voting])

                                    # Make a single dataframe will relevant information
                                    finalized_dataframe_loop = bill_votes_loop
                                    finalized_dataframe_loop['bill_id'] = df_loop['bill_id'][0]
                                    finalized_dataframe_loop['subjects_top_terms'] = df_loop['subjects_top_term'][0]
                                    finalized_dataframe['subjects'] = str(df['subjects'][0])
                                    finalized_dataframe_loop['date'] = df_loop['summary.date'][0]

                                    df = df.append(df_loop).reset_index(drop=True)
                                    bill_votes = bill_votes.append(bill_votes_loop).reset_index(drop=True)
                                    finalized_dataframe = finalized_dataframe.append(finalized_dataframe_loop).reset_index(drop=True)

                    except KeyError:
                        print "no summary date"
                else:
                    print "no roll data for hr%d" %i
            except IndexError:
                print "list index out of range"
if err.errno == 60:
    print "must sleep for 5 seconds"
    time.sleep(5)

Alex, you da man! Searching for hr1
Alex, you da man! Searching for hr1339
no roll data for hr1339
Alex, you da man! Searching for hr1340
no roll data for hr1340
Alex, you da man! Searching for hr1341
no roll data for hr1341
Alex, you da man! Searching for hr1342
no roll data for hr1342
Alex, you da man! Searching for hr1343
no roll data for hr1343
Alex, you da man! Searching for hr1344
no roll data for hr1344
Alex, you da man! Searching for hr1345
no roll data for hr1345
Alex, you da man! Searching for hr1346
no roll data for hr1346
Alex, you da man! Searching for hr1347
no roll data for hr1347
Alex, you da man! Searching for hr1348
no roll data for hr1348
Alex, you da man! Searching for hr1349
no roll data for hr1349
Alex, you da man! Searching for hr1350
no roll data for hr1350
Alex, you da man! Searching for hr1351
no roll data for hr1351
Alex, you da man! Searching for hr1352
no roll data for hr1352
Alex, you da man! Searching for hr1353
no roll data for hr1353
Alex, you da man! S

NameError: name 'err' is not defined

In [13]:
finalized_dataframe.to_excel("/Users/Alexanderhubbard/Desktop/congress_114_bill_votes_part_2.xlsx")
df.to_excel("/Users/Alexanderhubbard/Desktop/df_114_part_2.xlsx")
bill_votes.to_excel("/Users/Alexanderhubbard/Desktop/bill_votes_114_part_2.xlsx")